# Memory
## Setup

In [1]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

In [2]:
import { z } from "zod";

const contextSchema = z.object({
    first: z.string(),
    last: z.string(),
});
type Context = z.infer<typeof contextSchema>;

In [3]:
import { tool, Runtime } from "langchain";

const executeSQL = tool(async ({ query }, runtime: Runtime<Context>) => {
    const placeholders = query.match(/:(\w+)/g);
    if (!placeholders) {
        return await db.run(query);
    }

    const updatedQuery = placeholders.reduce((acc, placeholder) => {
        const key = placeholder.slice(1) as keyof Context;
        return acc.replace(placeholder, `"${runtime.context[key]}"`);
    }, query);

    return await db.run(updatedQuery);
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results. Named parameters like :first and :last will be filled from runtime context.",
    schema: z.object({
        query: z.string()
    })
});

In [4]:
import { tool } from "langchain";

const FALLBACK_RATES = { USD: 1.0, EUR: 0.92, JPY: 150.0, GBP: 0.79 }

const convertCurrency = tool(async ({ amount, toCurrency }) => {
    try {
        const url = `https://api.exchangerate.host/convert?from=USD&to=${toCurrency}&amount=${amount}`
        const response = await fetch(url, {
            headers: {
                "Accept": "application/json"
            }
        })
        const data = await response.json()
        const result = data.result
        if (result) {
            return Math.round(amount * FALLBACK_RATES[to_currency], 2)
        }
    } catch (error) {
        console.error(`[convert_currency] Falling back due to error: ${error}`)
    }
    if (!(toCurrency in FALLBACK_RATES)) {
        throw new Error("Unsupported currency in fallback mode")
    }
    return Math.round(amount * FALLBACK_RATES[toCurrency], 2)
}, {
    name: "convert_currency",
    description: `
        Convert an amount in USD to another currency using live exchange rates.
        Always use this tool if the user requests a currency different 
        from the one stored in the database (USD).
    `,
    schema: z.object({
        amount: z.number(),
        toCurrency: z.string()
    })
});

In [5]:
export const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.`

In [7]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [convertCurrency, executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    middleware: []
})

## Repeated Queries

In [8]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What was the total on my last invoice?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    streamMode: "values",
});

result.messages.at(-1).content;

"Your last invoice total was $5.94 (USD). It’s Invoice #374 dated 2013-07-04."

In [9]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What were the titles?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    streamMode: "values",
});

result.messages.at(-1).content;

"Could you clarify which titles you mean—album titles, track titles, playlist titles, or something else?"

## Add memory

In [11]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver()
const agent = createAgent({
    model: "openai:gpt-5",
    tools: [convertCurrency, executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    checkpointer,
    middleware: []
})

In [12]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What was the total on my last invoice?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    configurable: { thread_id: "1" },
    streamMode: "values",
});

result.messages.at(-1).content;

"Your most recent invoice total was $5.94 (USD)."

In [13]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What were the titles?");
const result = await agent.invoke({
    messages: [question]
}, {
    context: { first: "Frank", last: "Harris" },
    configurable: { thread_id: "1" },
    streamMode: "values",
});

result.messages.at(-1).content;

"The first five track titles on your most recent invoice are:\n" +
  "- Holier Than Thou\n" +
  "- Through The Never\n" +
  "- My Friend Of Misery\n" +
  "- The Wait\n" +
  "- Blitzkrieg\n" +
  "\n" +
  "Want me to list all items from that invoice?"